<a href="https://colab.research.google.com/github/bradymiller2310/FantasyFootballDashboard/blob/main/ESPN_FF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install espn-api
!pip install pandas
!pip install openpyxl

In [2]:
from espn_api.football import League
import pandas as pd
import numpy as np

In [3]:
league = League(league_id=1292514, year=2024, espn_s2="AECPNWyLvfnlcFfE0Xb73O%2BP5ZU9n1G%2FSnP3ixxvuqSwQRcQe7bEQfBuz3YBNeRluTYofdhX2aTajj0jZSF%2B8i9ewdk8Dwf6p9G215F2FjmI06wPKyhjleL4WzaT3PCiP15FDXe55YNMkQeJXgbsCuK1VHbdQ1Nxidq0JxAHezRn2zAt6WwNtSqrF0k2nNHV33VpJqPmMk4msFmaoCaz0UAp95xxy1YmXyEUnZWgA60HJkbsgYkZ8AaESwFENsPYdUvKYqCwnZLjvgI5varhOTDYekjvokdtaCoxJfzOdGLJ3g%3D%3D", swid="{D68FE5F2-46CE-421C-B248-DDA01360BB34}")
print(league.teams)  #Teams in league

[Team(Fightin' Furries), Team(Quon Solo), Team(Jen-eral ⚔️), Team(Captain Sweatpants), Team(Salmon LipBalm !!), Team(Graham’s Groupie), Team(pop-pop's bible study), Team(Bucktown Bandits), Team(bird gang), Team(bungalicious  💅)]


## **Testing out api functionalities**
*This section contains code chunks that load in various data - used to see what we had access to.*

In [ ]:
for week in range(1, 15):
    print(f"Week {week} Schedule:")
    for matchup in league.scoreboard(week=week):
        print(f"{matchup.home_team} vs {matchup.away_team}")

Week 1 Schedule:
Team(Bucktown Bandits) vs Team(Graham’s Groupie)
Team(bungalicious  💅) vs Team(Fightin' Furries)
Team(bird gang) vs Team(Salmon LipBalm !!)
Team(pop-pop's bible study) vs Team(Quon Solo)
Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
Week 2 Schedule:
Team(Fightin' Furries) vs Team(Salmon LipBalm !!)
Team(Quon Solo) vs Team(Bucktown Bandits)
Team(Captain Sweatpants) vs Team(Graham’s Groupie)
Team(pop-pop's bible study) vs Team(bungalicious  💅)
Team(Jen-eral ⚔️) vs Team(bird gang)
Week 3 Schedule:
Team(Quon Solo) vs Team(Captain Sweatpants)
Team(Fightin' Furries) vs Team(pop-pop's bible study)
Team(Salmon LipBalm !!) vs Team(Jen-eral ⚔️)
Team(Bucktown Bandits) vs Team(bungalicious  💅)
Team(Graham’s Groupie) vs Team(bird gang)
Week 4 Schedule:
Team(pop-pop's bible study) vs Team(Jen-eral ⚔️)
Team(bungalicious  💅) vs Team(Quon Solo)
Team(bird gang) vs Team(Captain Sweatpants)
Team(Bucktown Bandits) vs Team(Fightin' Furries)
Team(Graham’s Groupie) vs Team(Salmon LipBalm !!)


In [ ]:
free_agents = league.free_agents(size=20, position='QB')  # Get top 20 available QBs

print("Available Free Agents (QB):")
for player in free_agents:
    print(f"  - {player.name} ({player.proTeam})")

Available Free Agents (QB):
  - C.J. Stroud (HOU)
  - Tua Tagovailoa (MIA)
  - Anthony Richardson (IND)
  - Aaron Rodgers (None)
  - Kirk Cousins (ATL)
  - Bryce Young (CAR)
  - Michael Penix Jr. (ATL)
  - Trevor Lawrence (JAX)
  - Jameis Winston (CLE)
  - Derek Carr (NO)
  - Justin Fields (NYJ)
  - Joe Flacco (IND)
  - Deshaun Watson (CLE)
  - Cooper Rush (BAL)
  - Daniel Jones (IND)
  - Tom Brady (TB)
  - Mason Rudolph (PIT)
  - Mac Jones (SF)
  - Tommy DeVito (NYG)
  - Aidan O'Connell (LV)


In [ ]:
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    print("Roster:")
    for player in team.roster:
        print(f"  - {player.name} ({player.position})")


Team: Fightin' Furries
Roster:
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Joe Mixon (RB)
  - Patrick Mahomes (QB)
  - Dalton Kincaid (TE)
  - Zay Flowers (WR)
  - Jaxon Smith-Njigba (WR)
  - Gus Edwards (RB)
  - Matthew Stafford (QB)
  - Kareem Hunt (RB)
  - Broncos D/ST (D/ST)
  - David Njoku (TE)
  - Caleb Williams (QB)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo
Roster:
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - Marvin Harrison Jr. (WR)
  - Mike Evans (WR)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB)
  - Kyler Murray (QB)
  - J.K. Dobbins (RB)
  - Geno Smith (QB)
  - Chris Boswell (K)
  - Romeo Doubs (WR)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)
  - Josh Downs (WR)
  - Evan Engram (TE)

Team: Jen-eral ⚔️
Roster:
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Raheem Mostert (RB)
  - Jets D/ST (D/ST)
  - Pat Freiermuth (TE)
  

In [ ]:
week_number = 1
box_scores = league.box_scores(week=week_number)

for matchup in box_scores:
    print(f"\nMatchup: {matchup.home_team} vs {matchup.away_team}")
    print(f"  Home Score: {matchup.home_score}")
    print(f"  Away Score: {matchup.away_score}")


Matchup: Team(Bucktown Bandits) vs Team(Graham’s Groupie)
  Home Score: 73.0
  Away Score: 96.0

Matchup: Team(bungalicious  💅) vs Team(Fightin' Furries)
  Home Score: 101.0
  Away Score: 94.0

Matchup: Team(bird gang) vs Team(Salmon LipBalm !!)
  Home Score: 59.0
  Away Score: 101.0

Matchup: Team(pop-pop's bible study) vs Team(Quon Solo)
  Home Score: 79.0
  Away Score: 97.0

Matchup: Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
  Home Score: 82.0
  Away Score: 110.0


In [ ]:
# getting injury status of free agents (not on rosters)

free_agents = league.free_agents()
i=0
for player in free_agents:
  player_obj = free_agents[i]
  print(f"Player: {player_obj.name}, Injury Status: {player_obj.injuryStatus}, Injured: {player_obj.injured}")
  i+=1

Player: Ladd McConkey, Injury Status: ACTIVE, Injured: False
Player: Keenan Allen, Injury Status: ACTIVE, Injured: False
Player: Michael Pittman Jr., Injury Status: QUESTIONABLE, Injured: False
Player: Steelers D/ST, Injury Status: [], Injured: False
Player: C.J. Stroud, Injury Status: ACTIVE, Injured: False
Player: Kyle Pitts, Injury Status: ACTIVE, Injured: False
Player: Adam Thielen, Injury Status: ACTIVE, Injured: False
Player: Chris Olave, Injury Status: QUESTIONABLE, Injured: False
Player: Tucker Kraft, Injury Status: ACTIVE, Injured: False
Player: 49ers D/ST, Injury Status: [], Injured: False
Player: Jerome Ford, Injury Status: QUESTIONABLE, Injured: False
Player: Texans D/ST, Injury Status: [], Injured: False
Player: Quentin Johnston, Injury Status: ACTIVE, Injured: False
Player: Tua Tagovailoa, Injury Status: QUESTIONABLE, Injured: False
Player: Nick Chubb, Injury Status: QUESTIONABLE, Injured: False
Player: Anthony Richardson, Injury Status: QUESTIONABLE, Injured: False
Playe

In [ ]:
# getting injury status of players on rosters
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    for player in team.roster:
        print(f"Player: {player.name}, Injury Status: {player.injuryStatus}, Injured: {player.injured}")


Team: Fightin' Furries
Player: Tyreek Hill, Injury Status: QUESTIONABLE, Injured: False
Player: Isiah Pacheco, Injury Status: ACTIVE, Injured: False
Player: Joe Mixon, Injury Status: ACTIVE, Injured: False
Player: Patrick Mahomes, Injury Status: ACTIVE, Injured: False
Player: Dalton Kincaid, Injury Status: ACTIVE, Injured: False
Player: Zay Flowers, Injury Status: QUESTIONABLE, Injured: False
Player: Jaxon Smith-Njigba, Injury Status: ACTIVE, Injured: False
Player: Gus Edwards, Injury Status: ACTIVE, Injured: False
Player: Matthew Stafford, Injury Status: ACTIVE, Injured: False
Player: Kareem Hunt, Injury Status: ACTIVE, Injured: False
Player: Broncos D/ST, Injury Status: NORMAL, Injured: False
Player: David Njoku, Injury Status: QUESTIONABLE, Injured: False
Player: Caleb Williams, Injury Status: QUESTIONABLE, Injured: False
Player: DeAndre Hopkins, Injury Status: ACTIVE, Injured: False
Player: Bills D/ST, Injury Status: NORMAL, Injured: False
Player: Matthew Wright, Injury Status: AC

In [ ]:
# getting starting roster for each week

for team in league.teams:
    print(f"\nTeam: {team.team_name} (Week {week})")

    # Get the team's roster for the given week
    for player in team.roster:
        #if player.lineupSlot != "BE" and player.lineupSlot != "IR":  # Exclude Bench and IR players
            print(f"  - {player.name} ({player.lineupSlot})")



Team: Fightin' Furries (Week 14)
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Joe Mixon (BE)
  - Patrick Mahomes (QB)
  - Dalton Kincaid (BE)
  - Zay Flowers (BE)
  - Jaxon Smith-Njigba (RB/WR/TE)
  - Gus Edwards (BE)
  - Matthew Stafford (BE)
  - Kareem Hunt (RB)
  - Broncos D/ST (BE)
  - David Njoku (TE)
  - Caleb Williams (BE)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo (Week 14)
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - Marvin Harrison Jr. (BE)
  - Mike Evans (BE)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB/WR/TE)
  - Kyler Murray (QB)
  - J.K. Dobbins (BE)
  - Geno Smith (BE)
  - Chris Boswell (K)
  - Romeo Doubs (BE)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)
  - Josh Downs (BE)
  - Evan Engram (BE)

Team: Jen-eral ⚔️ (Week 14)
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Raheem Mostert (BE)
  - Jets D/ST (D/ST)
  - Pat Fr

## **Getting the fantasy team rosters**
*Will use to join to player data to maintain team assigments*

In [4]:
# Define the range of weeks
weeks = range(1, 19)  # Weeks 1 to 18

# Create an empty list to store data
all_weeks_roster = []

# Loop through each week and get the roster data
for week in weeks:
    week_data = []  # Temporary storage for current week's data

    for team in league.teams:
        team_name = team.team_name  # Fantasy Team Name

        for player in team.roster:
            week_data.append({
                "cur_Fteam": team_name,
                "week": week,
                "espn_id": player.playerId,  # ESPN Unique Player ID
                "cur_roster_slot": player.position if player.lineupSlot != "BE" else "Bench"
            })

    # If no data is returned for this week, stop iterating
    if not week_data:
        print(f"No data found for Week {week}. Stopping iteration.")
        break  # Stop looping when a week has no data

    # Add this week's data to the master list
    all_weeks_roster.extend(week_data)

# Convert to DataFrame
weekly_rosters = pd.DataFrame(all_weeks_roster)

cur_roster = weekly_rosters[weekly_rosters['week'] == weekly_rosters['week'].max()].drop('week', axis=1)
print(cur_roster)

             cur_Fteam  espn_id cur_roster_slot
2720  Fightin' Furries  3116406              WR
2721  Fightin' Furries  4361529              RB
2722  Fightin' Furries  3116385           Bench
2723  Fightin' Furries  3139477              QB
2724  Fightin' Furries  4385690           Bench
...                ...      ...             ...
2875   bungalicious  💅  4243537           Bench
2876   bungalicious  💅  4689936               K
2877   bungalicious  💅  3912547              QB
2878   bungalicious  💅   -16011            D/ST
2879   bungalicious  💅  4426385              RB

[160 rows x 3 columns]


### **Getting statistical data from xlsx files**

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [6]:
import pandas as pd

file_path = "/content/drive/My Drive/ff_data.xlsx"

# Load specific sheets
season_stats_all = pd.read_excel(file_path, sheet_name="Season", engine="openpyxl")
weekly_stats_all = pd.read_excel(file_path, sheet_name="Weekly", engine="openpyxl")

# Display first few rows
print(season_stats_all.head())
print(weekly_stats_all.head())

   season   player_id player_display_name team position  games_played  \
0    2021  00-0019596           Tom Brady   TB       QB          20.0   
1    2021  00-0022924  Ben Roethlisberger  PIT       QB          18.0   
2    2021  00-0023459       Aaron Rodgers   GB       QB          17.0   
3    2021  00-0023682    Ryan Fitzpatrick  WAS       QB           1.0   
4    2021  00-0024243      Marcedes Lewis   GB       TE          14.0   

   completions  attempts  comp%  passing_yards  ...  total_st_tds  st_tds_pg  \
0        573.0     847.0  67.65         6187.0  ...           NaN        NaN   
1        448.0     693.0  64.65         4170.0  ...           NaN        NaN   
2        386.0     560.0  68.93         4340.0  ...           NaN        NaN   
3          3.0       6.0  50.00           13.0  ...           NaN        NaN   
4          0.0       0.0    NaN            0.0  ...           NaN        NaN   

   total_fp  fp_pg  espn_id                name   age  height  weight  \
0      

## **Joining current fantasy team roster data with weekly and season data**

### *Season data*

In [7]:
season_stats_all = pd.merge(season_stats_all, cur_roster, on="espn_id", how="left")

season_stats_all.loc[
    (season_stats_all['season'] == 2024) &
    (season_stats_all['cur_Fteam'] != 'FA') &
    (season_stats_all['cur_roster_slot'] != 'FA'),
    ['cur_Fteam', 'cur_roster_slot']
] = 'FA'

In [ ]:
#season_stats_all['season'] = season_stats_all['season'].astype(str)

In [8]:
print(season_stats_all.head(10))

   season   player_id player_display_name      team position  games_played  \
0    2021  00-0019596           Tom Brady        TB       QB          20.0   
1    2021  00-0022924  Ben Roethlisberger       PIT       QB          18.0   
2    2021  00-0023459       Aaron Rodgers        GB       QB          17.0   
3    2021  00-0023682    Ryan Fitzpatrick       WAS       QB           1.0   
4    2021  00-0024243      Marcedes Lewis        GB       TE          14.0   
5    2021  00-0025394     Adrian Peterson  TEN, SEA       RB           4.0   
6    2021  00-0026035      Danny Amendola       HOU       WR           8.0   
7    2021  00-0026143           Matt Ryan       ATL       QB          17.0   
8    2021  00-0026158          Joe Flacco       NYJ       QB           2.0   
9    2021  00-0026189      DeSean Jackson    LA, LV       WR          16.0   

   completions  attempts  comp%  passing_yards  ...  total_fp  fp_pg  espn_id  \
0        573.0     847.0  67.65         6187.0  ...       Na

### *Weekly data*

In [9]:
weekly_stats_all = pd.merge(weekly_stats_all, weekly_rosters, on=["espn_id", "week"], how="left")

weekly_stats_all.loc[
    (weekly_stats_all['season'] == 2024) &
    (weekly_stats_all['cur_Fteam'] != 'FA') &
    (weekly_stats_all['cur_roster_slot'] != 'FA'),
    ['cur_Fteam', 'cur_roster_slot']
] = 'FA'

In [ ]:
#weekly_stats_all['week'] = weekly_stats_all['week'].astype(str)
#weekly_stats_all['season'] = weekly_stats_all['season'].astype(str)

In [11]:
print(weekly_stats_all.head(10))

    player_id player_name player_display_name position position_group  \
0  00-0019596     T.Brady           Tom Brady       QB             QB   
1  00-0019596     T.Brady           Tom Brady       QB             QB   
2  00-0019596     T.Brady           Tom Brady       QB             QB   
3  00-0019596     T.Brady           Tom Brady       QB             QB   
4  00-0019596     T.Brady           Tom Brady       QB             QB   
5  00-0019596     T.Brady           Tom Brady       QB             QB   
6  00-0019596     T.Brady           Tom Brady       QB             QB   
7  00-0019596     T.Brady           Tom Brady       QB             QB   
8  00-0019596     T.Brady           Tom Brady       QB             QB   
9  00-0019596     T.Brady           Tom Brady       QB             QB   

                                        headshot_url recent_team  season  \
0  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
1  https://static.www.nfl.com/image/private/

# **Getting matchup data**

In [10]:
total_weeks = league.settings.reg_season_count  # Regular season length

# Initialize records dict
team_records = {team.team_name: {'W': 0, 'L': 0} for team in league.teams}

# Store all weekly matchups and records
matchups_all_weeks = []

# Loop through each week
for week in range(1, total_weeks + 1):
    scoreboard = league.scoreboard(week=week)

    for matchup in scoreboard:
        home = matchup.home_team.team_name
        away = matchup.away_team.team_name
        home_score = matchup.home_score
        away_score = matchup.away_score

        # Determine winner and update records
        if home_score > away_score:
            team_records[home]['W'] += 1
            team_records[away]['L'] += 1
        elif home_score < away_score:
            team_records[away]['W'] += 1
            team_records[home]['L'] += 1
        else:
            # Tie case (optional)
            pass

        # Record after this week
        home_record = f"{team_records[home]['W']}-{team_records[home]['L']}"
        away_record = f"{team_records[away]['W']}-{team_records[away]['L']}"

        matchups_all_weeks.append({
            'week': week,
            'home_team': home,
            'away_team': away,
            'home_score': home_score,
            'away_score': away_score,
            'home_record': home_record,
            'away_record': away_record
        })

# Convert to DataFrame
weekly_matchup_data = pd.DataFrame(matchups_all_weeks)

# Display the result
print(weekly_matchup_data)

    week              home_team              away_team  home_score  \
0      1       Bucktown Bandits       Graham’s Groupie        73.0   
1      1        bungalicious  💅       Fightin' Furries       101.0   
2      1              bird gang      Salmon LipBalm !!        59.0   
3      1  pop-pop's bible study              Quon Solo        79.0   
4      1            Jen-eral ⚔️     Captain Sweatpants        82.0   
..   ...                    ...                    ...         ...   
65    14       Graham’s Groupie       Bucktown Bandits       143.0   
66    14      Salmon LipBalm !!       Fightin' Furries       115.0   
67    14     Captain Sweatpants              Quon Solo        88.0   
68    14            Jen-eral ⚔️  pop-pop's bible study        68.0   
69    14              bird gang        bungalicious  💅        81.0   

    away_score home_record away_record  
0         96.0         0-1         1-0  
1         94.0         1-0         0-1  
2        101.0         0-1         1

# **Getting injury status**
PROBABLY CAN'T USE B/C ONLY HAVE 2024 DATA

In [ ]:
all_players = []
current_week = league.current_week

# 1. Rostered players from each team
for team in league.teams:
    for player in team.roster:
        all_players.append({
            "week": current_week,
            "team_owner": team.team_name,
            "name": player.name,
            "position": player.position,
            "espn_id": player.playerId,
            "injured": player.injured,
            "injury_status": player.injuryStatus,
            "rostered": True
        })

# 2. Free agents (unrostered players)
for player in league.free_agents(size=1000):  # you can adjust the size to pull more players
    all_players.append({
        "week": current_week,
        "team_owner": "Free Agent",
        "name": player.name,
        "position": player.position,
        "espn_id": player.playerId,
        "injured": player.injured,
        "injury_status": player.injuryStatus,
        "rostered": False
    })

# Optional: Convert to pandas DataFrame
import pandas as pd
injury_status = pd.DataFrame(all_players)

# Display or export
print(injury_status.head(10))

print(injury_status[injury_status["team_owner"] == "Free Agent"].head(10))

   week        team_owner                name position  espn_id  injured  \
0    17  Fightin' Furries         Tyreek Hill       WR  3116406    False   
1    17  Fightin' Furries       Isiah Pacheco       RB  4361529    False   
2    17  Fightin' Furries           Joe Mixon       RB  3116385    False   
3    17  Fightin' Furries     Patrick Mahomes       QB  3139477    False   
4    17  Fightin' Furries      Dalton Kincaid       TE  4385690    False   
5    17  Fightin' Furries         Zay Flowers       WR  4429615    False   
6    17  Fightin' Furries  Jaxon Smith-Njigba       WR  4430878    False   
7    17  Fightin' Furries         Gus Edwards       RB  3051926    False   
8    17  Fightin' Furries    Matthew Stafford       QB    12483    False   
9    17  Fightin' Furries         Kareem Hunt       RB  3059915    False   

  injury_status  rostered  
0  QUESTIONABLE      True  
1        ACTIVE      True  
2        ACTIVE      True  
3        ACTIVE      True  
4        ACTIVE      Tr

# **QB CatBoost Model**

## **Feature Engineering**
*Need to create features that can be used in both train and test sets since we can't use rush yards, pass yards (any game specific stats) because they have not occurred and need to use like rolling features or averages*

In [17]:
df_qb = weekly_stats_all[weekly_stats_all['position'] == 'QB'].copy()

# -------------------------------
# STEP 2: Sort for time-series ops
# -------------------------------
df_qb = df_qb.sort_values(by=['player_id', 'season', 'week'])

# -------------------------------
# STEP 3: Lag + Rolling Features (3-game window)
# -------------------------------
rolling_cols = [
    'passing_yards', 'passing_tds', 'interceptions', 'passing_epa',
    'rushing_yards', 'rushing_tds', 'completions', 'attempts',
    'fantasy_points_ppr', 'passing_air_yards', 'passing_yards_after_catch',
    'rushing_fumbles_lost', 'rushing_epa', 'rushing_2pt_conversions', 'passing_2pt_conversions'
]

for col in rolling_cols:
    df_qb[f'{col}_last3'] = df_qb.groupby('player_id')[col].shift(1).rolling(3, min_periods = 1).mean()

# -------------------------------
# STEP 4: Expanding (season-to-date) averages
# -------------------------------
expanding_cols = ['passing_yards', 'passing_tds', 'interceptions', 'rushing_yards', 'rushing_tds', 'fantasy_points_ppr'
]

for col in expanding_cols:
    df_qb[col] = pd.to_numeric(df_qb[col], errors='coerce')

for col in expanding_cols:
    df_qb[f'{col}_season_avg'] = (
        df_qb.groupby(['player_id', 'season'])[col]
        .transform(lambda x: x.shift(1).expanding().mean())
    )

df_qb['completions'] = pd.to_numeric(df_qb['completions'], errors='coerce')
df_qb['attempts'] = pd.to_numeric(df_qb['attempts'], errors='coerce')


df_qb['yards_per_attempt'] = df_qb['passing_yards'] / df_qb['attempts'].replace(0, pd.NA)

df_qb['comp%'] = (
    (df_qb['completions'] / df_qb['attempts']) * 100
).replace([pd.NA, np.inf, -np.inf], np.nan)

df_qb['comp%'] = pd.to_numeric(df_qb['comp%'], errors='coerce')

df_qb['yards_per_attempt'] = pd.to_numeric(df_qb['yards_per_attempt'], errors='coerce')

print(df_qb.head(10))

    player_id player_name player_display_name position position_group  \
0  00-0019596     T.Brady           Tom Brady       QB             QB   
1  00-0019596     T.Brady           Tom Brady       QB             QB   
2  00-0019596     T.Brady           Tom Brady       QB             QB   
3  00-0019596     T.Brady           Tom Brady       QB             QB   
4  00-0019596     T.Brady           Tom Brady       QB             QB   
5  00-0019596     T.Brady           Tom Brady       QB             QB   
6  00-0019596     T.Brady           Tom Brady       QB             QB   
7  00-0019596     T.Brady           Tom Brady       QB             QB   
8  00-0019596     T.Brady           Tom Brady       QB             QB   
9  00-0019596     T.Brady           Tom Brady       QB             QB   

                                        headshot_url recent_team  season  \
0  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
1  https://static.www.nfl.com/image/private/

In [18]:
# -------------------------------
# STEP 5: Efficiency metrics
# -------------------------------
df_qb['pass_attempts_shifted'] = df_qb.groupby('player_id')['attempts'].shift(1)

df_qb['comp_pct_last3'] = df_qb.groupby('player_id')['comp%'].shift(1).rolling(3,min_periods = 1).mean()
df_qb['yards_per_attempt_last3'] = df_qb.groupby('player_id')['yards_per_attempt'].shift(1).rolling(3,min_periods = 1).mean()
df_qb['td_rate_last3'] = df_qb.groupby('player_id')['passing_tds'].shift(1).rolling(3,min_periods = 1).sum() / df_qb['pass_attempts_shifted']
df_qb['int_rate_last3'] = df_qb.groupby('player_id')['interceptions'].shift(1).rolling(3,min_periods = 1).sum() / df_qb['pass_attempts_shifted']

# -------------------------------
# STEP 6: Advanced metrics
# -------------------------------
advanced = ['pacr', 'dakota']
for col in advanced:
    df_qb[f'{col}_trend'] = df_qb.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 7: Opponent Defense (already lagged by design)
# -------------------------------
opponent_cols = ['opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg', 'opp_sacks_pg', 'opp_fumbles_pg']
for col in opponent_cols:
    df_qb[f'{col}_trend'] = df_qb.groupby('player_id')[col].shift(1).rolling(3,min_periods = 1).mean()

# -------------------------------
# STEP 8: Clean up NA rows (from shifting/rolling)
# -------------------------------
#df_qb = df_qb.dropna()

In [37]:
print(df_qb.head(10))

    player_id player_name player_display_name position position_group  \
0  00-0019596     T.Brady           Tom Brady       QB             QB   
1  00-0019596     T.Brady           Tom Brady       QB             QB   
2  00-0019596     T.Brady           Tom Brady       QB             QB   
3  00-0019596     T.Brady           Tom Brady       QB             QB   
4  00-0019596     T.Brady           Tom Brady       QB             QB   
5  00-0019596     T.Brady           Tom Brady       QB             QB   
6  00-0019596     T.Brady           Tom Brady       QB             QB   
7  00-0019596     T.Brady           Tom Brady       QB             QB   
8  00-0019596     T.Brady           Tom Brady       QB             QB   
9  00-0019596     T.Brady           Tom Brady       QB             QB   

                                        headshot_url recent_team  season  \
0  https://static.www.nfl.com/image/private/f_aut...          TB    2021   
1  https://static.www.nfl.com/image/private/

***Getting list of columns to select from***

In [42]:
print(df_qb.columns.tolist())

['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'recent_team', 'season', 'week', 'season_type', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr', 'half_PPR', 'depth_pos', 'offense_snaps', 'offense_pct', 'opp_avg_ypg_al

In [19]:
TARGET_COL = 'fantasy_points_ppr'

train_df = df_qb[
    (df_qb[TARGET_COL].notna()) &
    (
        (df_qb['season'] < 2024) |
        ((df_qb['season'] == 2024) & (df_qb['week'] < 14))
    )
]

test_df = df_qb[(df_qb['season'] == 2024) & (df_qb['week'] == 14)]


feature_cols = [
    col for col in df_qb.columns
    if col.endswith('_last3') or col.endswith('_trend') or col.endswith('_season_avg')
] + ['player_display_name','season', 'week', 'opponent_team']


X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL]

X_test = test_df[feature_cols]

In [68]:
#!pip install --upgrade numpy

#!pip uninstall -y catboost
!pip install catboost --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
Found existing installation: catboost 1.2.7
Uninstalling catboost-1.2.7:
  Successfully uninstalled catboost-1.2.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
from catboost import CatBoostRegressor, Pool

In [24]:
categorical_cols = ['opponent_team', 'season', 'week', 'player_display_name']

# Create Pool objects
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_cols)
test_pool = Pool(data=X_test, cat_features=categorical_cols)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    loss_function='RMSEWithUncertainty',
    random_seed=42,
    verbose=50  # or 0 to suppress output
)

# Train
model.fit(train_pool)

0:	learn: 3.5902913	total: 16.5ms	remaining: 3.28s
50:	learn: 3.2442035	total: 1.86s	remaining: 5.44s
100:	learn: 3.1435875	total: 3.2s	remaining: 3.13s
150:	learn: 3.0344682	total: 4.16s	remaining: 1.35s
199:	learn: 2.9331765	total: 5.12s	remaining: 0us


In [26]:
y_pred = model.predict(test_pool, prediction_type='TotalUncertainty')

# Attach predictions to test set for review
#X_test_copy = X_test.copy()
#X_test_copy['predicted_fantasy_points'] = y_pred

#print(X_test_copy[['season', 'week', 'opponent_team', 'predicted_fantasy_points', 'player_display_name']])

CatBoostError: Invalid value of prediction_type=TotalUncertainty: must be Class, RawFormulaVal, Probability, LogProbability, Exponent, RMSEWithUncertainty.

In [ ]:
preds = np.array(y_pred)
y_pred = preds[:, 0]
std_dev = preds[:, 1]

ci_upper = y_pred + 1.645 * std_dev
ci_lower = y_pred - 1.645 * std_dev

In [ ]:
X_test_copy = X_test.copy()
X_test_copy['predicted_fp'] = y_pred
X_test_copy['ci_lower_90'] = ci_lower
X_test_copy['ci_upper_90'] = ci_upper

# Optional: define boom/bust
X_test_copy['result_type'] = 'neutral'  # default
X_test_copy.loc[X_test_copy['fantasy_points_ppr'] > X_test_copy['ci_upper_90'], 'result_type'] = 'boom'
X_test_copy.loc[X_test_copy['fantasy_points_ppr'] < X_test_copy['ci_lower_90'], 'result_type'] = 'bust'

### *old*

In [ ]:
train_qb = weekly_stats_all[
    ((weekly_stats_all['season'].astype(int) < 2024) |
     ((weekly_stats_all['season'].astype(int) == 2024) & (weekly_stats_all['week'].astype(int) < 14))) &
    (weekly_stats_all['position'] == 'QB')
].copy()

# Test: only QBs from Week 14 of 2024
test_qb = weekly_stats_all[
    (weekly_stats_all['season'].astype(int) == 2024) &
    (weekly_stats_all['week'].astype(int) == 14) &
    (weekly_stats_all['position'] == 'QB')
].copy()

test_qb = test_qb.dropna(subset=["fantasy_points"])

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, r2_score

**Defining features to be used**

In [ ]:
categorical_features = ['team', 'opponent_team']  # these are column names, not indices
numerical_features = features = [
    'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
    'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost',
    'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs',
    'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota',
    'carries', 'rushing_yards', 'rushing_tds',
    'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
    'rushing_epa', 'rushing_2pt_conversions',
    'racr', 'wopr',
    'offense_snaps', 'offense_pct',
    'opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg',
    'opp_fumbles_pg', 'opp_df_tds_pg', 'opp_sacks_pg', 'age', 'week', 'season'
] # adjust as needed

features = categorical_features + numerical_features
target = 'fantasy_points'

**Creating test and train sets**

In [ ]:
X_train = train_qb[features]
y_train = train_qb[target]

X_test = test_qb[features]
y_test = test_qb[target]

In [ ]:
# Drop rows where the target is NaN
train_mask = ~y_train.isna()
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = ~y_test.isna()
X_test = X_test[test_mask]
y_test = y_test[test_mask]

**Training the model**

In [ ]:
for col in categorical_features:
    X_train[col] = X_train[col].fillna("NA").astype(str)
    X_test[col] = X_test[col].fillna("NA").astype(str)

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, y_test, cat_features=categorical_features)

# 7. Train the model
model = CatBoostRegressor(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50
)

model.fit(train_pool)

0:	learn: 8.6098981	total: 5.42ms	remaining: 1.62s
50:	learn: 2.1120422	total: 264ms	remaining: 1.29s
100:	learn: 1.1098378	total: 517ms	remaining: 1.02s
150:	learn: 0.8242850	total: 780ms	remaining: 769ms
200:	learn: 0.6784543	total: 1.02s	remaining: 504ms
250:	learn: 0.5612642	total: 1.28s	remaining: 249ms
299:	learn: 0.4891954	total: 1.51s	remaining: 0us


**Testing and evaluating the model**

In [ ]:
# 8. Predict and evaluate
y_pred = model.predict(test_pool)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE on Week 14 of 2024: {mae:.2f}")
print(f"R² Score: {r2:.3f}")

# 9. Attach predictions to test data (optional)
test_qb['predicted_fp'] = y_pred

MAE on Week 14 of 2024: 0.67
R² Score: 0.972


In [ ]:
qb_pred_results = test_qb[[
    'player_display_name',
    'position',
    'team',
    'fantasy_points'
]].copy()

qb_pred_results['proj_fp'] = test_qb['predicted_fp']

# (Optional) Sort by projected points
qb_pred_results = qb_pred_results.sort_values(by='proj_fp', ascending=False)

# Show top rows
print(qb_pred_results)

      player_display_name position team  fantasy_points    proj_fp
18457          Josh Allen       QB  NaN           51.88  42.036743
18484         Sam Darnold       QB  NaN           34.58  33.663590
19480          Joe Burrow       QB  NaN           24.56  25.283274
19359         Jalen Hurts       QB  NaN           24.22  24.771326
19093      Tua Tagovailoa       QB  NaN           23.54  24.391290
20581         Brock Purdy       QB  NaN           22.10  22.612483
17510          Jared Goff       QB  NaN           22.02  22.472872
16933    Matthew Stafford       QB  NaN           20.80  21.276920
18439      Baker Mayfield       QB  NaN           19.00  19.034256
18626        Kyler Murray       QB  NaN           17.96  17.665650
16904       Aaron Rodgers       QB  NaN           17.56  17.589008
16953      Russell Wilson       QB  NaN           16.02  16.184173
18952           Drew Lock       QB  NaN           14.98  15.340196
17192      Jameis Winston       QB  NaN           15.28  15.07

# **RB CatBoost Model**

In [ ]:
train_rb = weekly_stats_all[
    ((weekly_stats_all['season'].astype(int) < 2024) |
     ((weekly_stats_all['season'].astype(int) == 2024) & (weekly_stats_all['week'].astype(int) < 14))) &
    (weekly_stats_all['position'] == 'RB')
].copy()

# Test: only QBs from Week 14 of 2024
test_rb = weekly_stats_all[
    (weekly_stats_all['season'].astype(int) == 2024) &
    (weekly_stats_all['week'].astype(int) == 14) &
    (weekly_stats_all['position'] == 'RB')
].copy()

test_rb = test_rb.dropna(subset=["fantasy_points"])

In [ ]:
categorical_features = ['team', 'opponent_team', 'depth_pos']  # these are column names, not indices
numerical_features = [
    # Rushing
    'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
    'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',

    # Receiving
    'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles',
    'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions',

    # Advanced receiving usage (optional but helpful)
    'racr', 'target_share', 'air_yards_share', 'wopr',

    # Snap info
    'offense_snaps', 'offense_pct',

    # Opponent context
    'opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg',
    'opp_fumbles_pg', 'opp_df_tds_pg', 'opp_sacks_pg'
] # adjust as needed

features = categorical_features + numerical_features
target = 'fantasy_points'

In [ ]:
X_train = train_rb[features]
y_train = train_rb[target]

X_test = test_rb[features]
y_test = test_rb[target]

In [ ]:
# Drop rows where the target is NaN
train_mask = ~y_train.isna()
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = ~y_test.isna()
X_test = X_test[test_mask]
y_test = y_test[test_mask]

In [ ]:
for col in categorical_features:
    X_train[col] = X_train[col].fillna("NA").astype(str)
    X_test[col] = X_test[col].fillna("NA").astype(str)

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, y_test, cat_features=categorical_features)

# 7. Train the model
model = CatBoostRegressor(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50
)

model.fit(train_pool)

0:	learn: 6.6113045	total: 10.3ms	remaining: 3.09s
50:	learn: 1.3104322	total: 355ms	remaining: 1.73s
100:	learn: 0.6548834	total: 848ms	remaining: 1.67s
150:	learn: 0.5279546	total: 1.66s	remaining: 1.64s
200:	learn: 0.4722498	total: 2.42s	remaining: 1.19s
250:	learn: 0.4319467	total: 3.27s	remaining: 638ms
299:	learn: 0.3979848	total: 4.09s	remaining: 0us


In [ ]:
# 8. Predict and evaluate
y_pred = model.predict(test_pool)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE on Week 14 of 2024: {mae:.2f}")
print(f"R² Score: {r2:.3f}")

# 9. Attach predictions to test data (optional)
test_rb['predicted_fp'] = y_pred

MAE on Week 14 of 2024: 0.23
R² Score: 0.998


In [ ]:
rb_pred_results = test_rb[[
    'player_display_name',
    'position',
    'team',
    'fantasy_points'
]].copy()

rb_pred_results['proj_fp'] = test_rb['predicted_fp']

# (Optional) Sort by projected points
rb_pred_results = rb_pred_results.sort_values(by='proj_fp', ascending=False)

# Show top rows
print(rb_pred_results)

         player_display_name position team  fantasy_points    proj_fp
21716        Zach Charbonnet       RB  NaN            31.3  31.425016
21867         Isaac Guerendo       RB  NaN            24.8  24.422743
20181          Rachaad White       RB  NaN            22.9  23.740302
18943            Josh Jacobs       RB  NaN            24.6  23.374129
20632         Kyren Williams       RB  NaN            21.7  22.315505
...                      ...      ...  ...             ...        ...
19485        Ke'Shawn Vaughn       RB  NaN             0.4   0.382264
17290         Ameer Abdullah       RB  NaN            -0.2   0.093578
21733          George Holani       RB  NaN            -0.1   0.039263
21797        Jonathon Brooks       RB  NaN            -0.3  -0.208935
17108  Cordarrelle Patterson       RB  NaN            -1.9  -1.419593

[63 rows x 5 columns]


# **WR CatBoost Model**

In [ ]:
train_wr = weekly_stats_all[
    ((weekly_stats_all['season'].astype(int) < 2024) |
     ((weekly_stats_all['season'].astype(int) == 2024) & (weekly_stats_all['week'].astype(int) < 14))) &
    (weekly_stats_all['position'] == 'WR')
].copy()

# Test: only QBs from Week 14 of 2024
test_wr = weekly_stats_all[
    (weekly_stats_all['season'].astype(int) == 2024) &
    (weekly_stats_all['week'].astype(int) == 14) &
    (weekly_stats_all['position'] == 'WR')
].copy()

test_wr = test_wr.dropna(subset=["fantasy_points"])

In [ ]:
categorical_features = ['team', 'opponent_team', 'depth_pos']  # these are column names, not indices
numerical_features = [
    # Rushing
    'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
    'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',

    # Receiving
    'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles',
    'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions',

    # Advanced receiving usage (optional but helpful)
    'racr', 'target_share', 'air_yards_share', 'wopr',

    # Snap info
    'offense_snaps', 'offense_pct',

    # Opponent context
    'opp_avg_ypg_allowed', 'opp_avg_ppg_allowed', 'opp_int_pg',
    'opp_fumbles_pg', 'opp_df_tds_pg', 'opp_sacks_pg'
] # adjust as needed

features = categorical_features + numerical_features
target = 'fantasy_points'

In [ ]:
X_train = train_wr[features]
y_train = train_wr[target]

X_test = test_wr[features]
y_test = test_wr[target]

# Drop rows where the target is NaN
train_mask = ~y_train.isna()
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = ~y_test.isna()
X_test = X_test[test_mask]
y_test = y_test[test_mask]

In [ ]:
for col in categorical_features:
    X_train[col] = X_train[col].fillna("NA").astype(str)
    X_test[col] = X_test[col].fillna("NA").astype(str)

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, y_test, cat_features=categorical_features)

# 7. Train the model
model = CatBoostRegressor(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=50
)

model.fit(train_pool)

0:	learn: 5.4906273	total: 10.8ms	remaining: 3.23s
50:	learn: 0.9326442	total: 460ms	remaining: 2.25s
100:	learn: 0.5048471	total: 919ms	remaining: 1.81s
150:	learn: 0.4234077	total: 1.35s	remaining: 1.33s
200:	learn: 0.3925521	total: 1.79s	remaining: 882ms
250:	learn: 0.3714056	total: 2.2s	remaining: 430ms
299:	learn: 0.3553020	total: 2.62s	remaining: 0us


In [ ]:
# 8. Predict and evaluate
y_pred = model.predict(test_pool)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE on Week 14 of 2024: {mae:.2f}")
print(f"R² Score: {r2:.3f}")

# 9. Attach predictions to test data (optional)
test_wr['predicted_fp'] = y_pred

MAE on Week 14 of 2024: 0.17
R² Score: 0.997


In [ ]:
wr_pred_results = test_wr[[
    'player_display_name',
    'position',
    'team',
    'fantasy_points'
]].copy()

wr_pred_results['proj_fp'] = test_wr['predicted_fp']

# (Optional) Sort by projected points
wr_pred_results = wr_pred_results.sort_values(by='proj_fp', ascending=False)

# Show top rows
print(wr_pred_results)

      player_display_name position team  fantasy_points    proj_fp
21407      Jordan Addison       WR  NaN            31.3  31.171067
19782       Ja'Marr Chase       WR  NaN            31.1  28.976821
21599          Puka Nacua       WR  NaN            29.8  27.164827
19262    Justin Jefferson       WR  NaN            25.2  25.184182
19168      Jauan Jennings       WR  NaN            21.0  20.930067
...                   ...      ...  ...             ...        ...
21534      Jonathan Mingo       WR  NaN             0.0   0.021821
21642         Jayden Reed       WR  NaN             0.0   0.020243
17412   Demarcus Robinson       WR  NaN             0.0   0.019475
22251    Malik Washington       WR  NaN             0.0   0.016025
21342         Jalin Hyatt       WR  NaN             0.0   0.013676

[95 rows x 5 columns]
